In [1]:
import pandas as pd
import numpy as np
import ast

In [128]:
df = pd.read_csv('./data/reviews.csv')

C:\Users\raxmo\AppData\Local\Temp\ipykernel_16416\3364534802.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/reviews.csv')


In [129]:
df = df.drop(['image', 'reviewTime', 'reviewerName', 'summary',
              'vote', 'unixReviewTime'], axis=1)
df.rename(columns={'overall': 'rating', 'asin': 'movie_id', 'reviewerID': 'user_id',
                   'reviewText':'reviews'}, inplace=True)
df.head()

,rating,verified,user_id,movie_id,style,reviews
0,5,True,A3478QRKQDOPQ2,0001527665,{'Format:': ' VHS Tape'},really happy they got evangelised .. spoiler a...
1,5,True,A2VHSG6TZHU1OB,0001527665,{'Format:': ' Amazon Video'},Having lived in West New Guinea (Papua) during...
2,5,False,A23EJWOW1TLENE,0001527665,{'Format:': ' Amazon Video'},Excellent look into contextualizing the Gospel...
3,5,True,A1KM9FNEJ8Q171,0001527665,{'Format:': ' Amazon Video'},"More than anything, I've been challenged to fi..."
4,4,True,A38LY2SSHVHRYB,0001527665,{'Format:': ' Amazon Video'},This is a great movie for a missionary going i...


In [130]:
rm_format = df['style'].apply(lambda x: isinstance(x, str) and "{'Format:" in x)
df = df.loc[rm_format]

df['style'] = df['style'].apply(lambda x: ast.literal_eval(x))
df['style'] = df['style'].apply(lambda x: x['Format:'])
df['style'] = df['style'].astype(str)

df = df[df['style'].isin(df['style'].value_counts()[df['style'].value_counts() >= 200000].index)]

In [131]:
df = df[df['verified'] == True]
df.drop(columns = ['verified', 'style'], inplace=True)

In [132]:
df.dropna(subset=['reviews'], inplace=True)
df.drop_duplicates(subset=['user_id', 'movie_id'], keep='first', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6212819 entries, 1 to 8765561
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
 3   reviews   object
dtypes: int64(1), object(3)
memory usage: 237.0+ MB


___

In [207]:
df2 = pd.read_csv('./data/reviews_meta.csv')
df2.info()

C:\Users\raxmo\AppData\Local\Temp\ipykernel_16416\444520879.py:1: DtypeWarning: Columns (1,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('./data/reviews_meta.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203766 entries, 0 to 203765
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   category         203766 non-null  object 
 1   tech1            6 non-null       object 
 2   description      203766 non-null  object 
 3   fit              0 non-null       float64
 4   title            203707 non-null  object 
 5   also_buy         203766 non-null  object 
 6   tech2            0 non-null       float64
 7   brand            137335 non-null  object 
 8   feature          203766 non-null  object 
 9   rank             203766 non-null  object 
 10  also_view        203766 non-null  object 
 11  main_cat         203756 non-null  object 
 12  similar_item     0 non-null       float64
 13  date             38 non-null      object 
 14  price            110745 non-null  object 
 15  asin             203766 non-null  object 
 16  imageURL         203766 non-null  obje

#### Dropping mostly null columns, and unecessary columns for modeling.

In [208]:
df2.rename(columns={'category': 'genre', 'brand': 'starring', 'asin': 'movie_id'}, inplace=True)
df2.drop_duplicates(subset='movie_id', inplace=True)
df2 = df2[df2['main_cat'] == 'Movies & TV']
df2.drop(['tech1', 'fit', 'tech2', 'similar_item',
         'date', 'price', 'imageURL', 'imageURLHighRes',
        'also_buy', 'also_view', 'feature', 'rank',
          'main_cat', 'details'], axis=1, inplace=True)

#### Reformating column types

In [209]:
df2['genre'] = df2['genre'].apply(lambda x: ast.literal_eval(x))

df2['description'] = df2['description'].apply(lambda x: " ".join(ast.literal_eval(x)).strip())
df2.loc[df2['description'].str.len() < 200, 'description'] = np.nan
df2.replace({'description': "###############################################################################################################################################################################################################################################################"}, np.nan, inplace=True)

In [210]:
# List of values to convert
convert = ['.', '\n', '-', '--', 'Na',
           'BRIDGESTONE MULTIMEDIA', '*', 'none',
           'na', 'N/a', 'VARIOUS', 'Artist Not Provided',
           'Sinister Cinema', 'Learn more', 'Various', 'various',
           'The Ambient Collection', 'Animation', 'Standard Deviants',
          'Animated']

# Apply 'Various Artists' to converted values and null values in 'starring' column
df2['starring'] = df2['starring'].apply(lambda x: 'Various Artists' if isinstance(x, str) and (x in convert or '\n' in x) else x)

# Fill remaining null values with 'Various Artists'
df2['starring'].fillna('Various Artists', inplace=True)

In [211]:
df2['genre'] = [x[1:] if len(x) > 1 and x[0] == 'Movies & TV' else x for x in df2['genre']]
df2 = df2[~df2['genre'].apply(lambda x: 'Exercise & Fitness' in x)]
df2.loc[df2['genre'].apply(lambda x: isinstance(x, list) and len(x) > 2 and x[0] == 'Art House & International'), 'genre'] = df2['genre'].apply(lambda x: [x[0] + ' ' + x[2]] if len(x) > 2 else x)
df2['genre'] = df2['genre'].apply(lambda x: x[:1] + x[2:] if isinstance(x, list) and len(x) > 2 and x[0] == 'Art House & International' and len(x) > 2 else x)

In [212]:
def remove_genre_from_list(lst):
    return [genre for genre in lst if genre not in remove_genre]

remove_genre = ['Genre for Featured Categories', 'Boxed Sets', 'All Titles', 
                'Independently Distributed', 'All Sony Pictures Titles', 'Studio Specials',
               'By Original Language', 'All BBC Titles', 'General',
               '20th Century Fox Home Entertainment', 'Family Features',
               'Lionsgate Home Entertainment', 'Fully Loaded DVDs', 'Blu-ray',
               'Holidays & Seasonal', 'All HBO Titles',
               'Paramount Home Entertainment', 'Music Artists', 'Walt Disney Studios Home Entertainment',
               'All', 'Universal Studios Home Entertainment'] 

df2['genre'] = df2['genre'].apply(lambda x: remove_genre_from_list(x) if isinstance(x, list) else x)
df2['genre'] = df2['genre'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
df2['genre'].replace({'': 'Unknown'}, inplace=True)

In [213]:
df2.dropna(subset='title', inplace=True)

In [214]:
df_col = df2.copy()
df_col.info()

<class 'pandas.core.frame.DataFrame'>
Index: 176166 entries, 0 to 203765
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   genre        176166 non-null  object
 1   description  117823 non-null  object
 2   title        176166 non-null  object
 3   starring     176166 non-null  object
 4   movie_id     176166 non-null  object
dtypes: object(5)
memory usage: 8.1+ MB


In [215]:
all_vid = df['movie_id'].unique().tolist()
df_col = df_col[df_col['movie_id'].isin(all_vid)]

In [216]:
all_vid2 = df_col['movie_id'].unique().tolist()
df_collab_rev = df[df['movie_id'].isin(all_vid2)]
df_collab_rev = df_collab_rev[df_collab_rev['user_id'].isin(df_collab_rev['user_id'].value_counts()[df_collab_rev['user_id'].value_counts() >= 5].index)]

In [217]:
all_vid3 = df_collab_rev['movie_id'].unique().tolist()
df_col = df_col[df_col['movie_id'].isin(all_vid3)]

In [218]:
df_collab_rev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2133511 entries, 21 to 8765560
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   rating    int64 
 1   user_id   object
 2   movie_id  object
 3   reviews   object
dtypes: int64(1), object(3)
memory usage: 81.4+ MB


In [112]:
df_collab_rev.to_csv('./data/collab_model_revs.csv',  encoding='utf-8', index=False)
df_col.to_csv('./data/collab_meta.csv',  encoding='utf-8', index=False)

In [219]:
df2.dropna(subset='description', inplace=True)

In [220]:
df2.drop_duplicates(subset='description', inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116955 entries, 2 to 203764
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   genre        116955 non-null  object
 1   description  116955 non-null  object
 2   title        116955 non-null  object
 3   starring     116955 non-null  object
 4   movie_id     116955 non-null  object
dtypes: object(5)
memory usage: 5.4+ MB


In [222]:
df2.to_csv('./data/desc_all_genre.csv', encoding='utf-8', index=False)

In [269]:
df3 = pd.read_csv('./data/desc_all_genre.csv')

In [362]:
df_genz = df3.copy()

In [363]:
not_genre = ['Movies','Special Interests','Sports','Sony Pictures Home Entertainment',
             'Warner Home Video','TV','MOD CreateSpace Video','All Lionsgate Titles',
             'A&E Home Video All A&E Titles','BBC','All Fox Titles','All Universal Studios Titles',
             'MGM Home Entertainment All MGM Titles','Unknown','DTS','HBO',
             'All Disney Titles','Fox TV','Special Editions','Miramax Home Entertainment',
             'Classics','Television','PBS', 'Top Sellers', 'New Yorker Films All New Yorker Titles',
             'yoga']

In [364]:
len(df_genz['genre'].value_counts())

571

In [365]:
df_genz = df_genz[~df_genz['genre'].isin(not_genre)]
df_genz.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72341 entries, 5 to 116954
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        72341 non-null  object
 1   description  72341 non-null  object
 2   title        72341 non-null  object
 3   starring     72341 non-null  object
 4   movie_id     72341 non-null  object
dtypes: object(5)
memory usage: 3.3+ MB


In [366]:
len(df_genz['genre'].value_counts())

546

In [367]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Anime.*', 'Anime', regex=True)
len(df_genz['genre'].value_counts())

544

In [368]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Animation.*', 'Animation', regex=True)
len(df_genz['genre'].value_counts())

537

In [369]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Musicals.*', 'Musicals & Performing Arts', regex=True)
df_genz['genre'] = df_genz['genre'].str.replace('.*Performing Arts.*', 'Musicals & Performing Arts', regex=True)
df_genz['genre'] = df_genz['genre'].str.replace('.*Music Video.*', 'Music Videos & Concerts', regex=True)
len(df_genz['genre'].value_counts())

525

In [370]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Science Fiction & Fantasy.*', 'Science Fiction & Fantasy', regex=True)
df_genz['genre'] = df_genz['genre'].str.replace('.*Sci-Fi & Fantasy.*', 'Science Fiction & Fantasy', regex=True)
df_genz['genre'] = df_genz['genre'].str.replace('.*Science Fiction.*', 'Science Fiction & Fantasy', regex=True)
df_genz['genre'] = df_genz['genre'].str.replace('.*Fantasy.*', 'Science Fiction & Fantasy', regex=True)
len(df_genz['genre'].value_counts())

506

In [371]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Art House.*', 'Art House & International', regex=True)
len(df_genz['genre'].value_counts())

452

In [372]:
df_genz['genre'] = df_genz['genre'].str.replace('Action', 'Action & Adventure', regex=True)
df_genz['genre'] = df_genz['genre'].str.replace('.*Action & Adventure.*', 'Action & Adventure', regex=True)
len(df_genz['genre'].value_counts())

438

In [373]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Christmas.*', 'Christmas', regex=True)
len(df_genz['genre'].value_counts())

432

In [374]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Christian Movies & TV.*', 'Faith & Spirituality', regex=True)
len(df_genz['genre'].value_counts())

431

In [375]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Reality TV.*', 'Reality TV', regex=True)
len(df_genz['genre'].value_counts())

430

In [376]:
df_genz['genre'] = df_genz['genre'].str.replace('.*Mystery.*', 'Mystery & Thrillers', regex=True)
len(df_genz['genre'].value_counts())

428

In [378]:
df_genz['genre'] = df_genz['genre'].str.replace('.*News.*', 'News', regex=True)
len(df_genz['genre'].value_counts())

419

In [381]:
df_genz = df_genz[df_genz['genre'].isin(df_genz['genre'].value_counts()[df_genz['genre'].value_counts() >= 210].index)]
df_genz.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67935 entries, 5 to 116954
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   genre        67935 non-null  object
 1   description  67935 non-null  object
 2   title        67935 non-null  object
 3   starring     67935 non-null  object
 4   movie_id     67935 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [385]:
df_genz.to_csv('./data/desc_filtered_genre.csv', encoding='utf-8', index=False)